# 1. Importing Libraries

In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
#!pip install --upgrade "watson-developer-cloud>=2.5.1"
from watson_developer_cloud import PersonalityInsightsV3

import json
from os.path import join

import pandas as pd
from pandas.io.json import json_normalize
from functools import reduce

import numpy as np

# 2.  Data Files: PDF to TXT

In [3]:
#!pip install pdfminer.six
#import pdfminer
#pdf2txt.py ycombinator-sam-altman.pdf > ycombinator-sam-altman.txt

In [4]:
#!git clone https://github.com/ssuleyma/InsideOut.git
path ='./InsideOut/Transcripts/'
files = os.listdir(path=path)
print(files)

['instagram-kevin-systrom.txt', 'facebook-mark-zuckerberg.txt', 'spotify-daniel-ek.txt', 'twitter-ev-williams.txt']


# 3. Analyze with Watson Personality Insights [DO NOT RUN]

<img src="https://www.easydna.ca/wp-content/uploads/2018/03/the_big_five1.png" alt="Big Five" style="width: 400px; height: 400px;"/>

In [5]:
pi_version = "2017-10-13"
pi_api_key = ""
pi_url = "https://gateway.watsonplatform.net/personality-insights/api"

personality_insights = PersonalityInsightsV3(
    version = pi_version,
    iam_apikey = pi_api_key,
    url = pi_url
)

personality_insights.set_detailed_response(True)
personality_insights.set_default_headers({'x-watson-learning-opt-out': "true"})

In [ ]:
candidates_df = pd.DataFrame()
for i,f in enumerate(files):
    with open(join(path, f)) as profile_txt:
        profile = personality_insights.profile(profile_txt.read(),
                                               content_type='text/plain',
                                               accept='application/json',
                                               consumption_preferences=True,
                                               raw_scores=True).get_result()
        
        candidates_df = candidates_df.append(json_normalize(profile),ignore_index=True,sort=False)
        candidates_df.loc[i,'file'] = f

In [ ]:
candidates_df.drop(columns=['processed_language','warnings'],inplace=True)

In [ ]:
candidates_df.head()

In [ ]:
# project.save_data('candidates_df.csv',candidates_df.to_csv(index=False),overwrite=True)

# 4. Data Cleaning

In [7]:
# The code was removed by Watson Studio for sharing.

,consumption_preferences,needs,personality,values,word_count,file
0,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2732,instagram-kevin-systrom.txt
1,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2517,facebook-mark-zuckerberg.txt
2,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",1854,spotify-daniel-ek.txt
3,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2852,twitter-ev-williams.txt


In [8]:
def insights_to_df(insight, candidates):
    """
    Returns a dataframe of needs, values or personality raw scores.
    
    insight - "needs", "values", "personality"
    candidates - dataframe of candidates
    """
    n_v_df = pd.DataFrame()
    for r in candidates.index:
        d = pd.DataFrame(eval(candidates.copy().loc[r,insight]))[['name','raw_score']]
        d = d.set_index('name').T.reset_index(drop=True)
        d['file'] = candidates.loc[r,'file']
        n_v_df = n_v_df.append(d,ignore_index=True)
    n_v_df.columns.name = ""
    return n_v_df

def consumption(candidates):
    """
    Returns a dataframe of positive consumption preferences of candidates.
    
    candidates - dataframe of candidates
    """
    c_df = pd.DataFrame()
    for r in candidates.index:
        d = pd.DataFrame(eval(candidates.copy().loc[r,'consumption_preferences'])).drop(columns=['consumption_preference_category_id'])
        d['consumption_preferences'] = d['consumption_preferences'].apply(lambda l: [p['consumption_preference_id'][24:] for p in l if p['score'] >= 0.5])
        d = d[~d['name'].isin(['Music Preferences','Purchasing Preferences'])]
        d = d.set_index('name').T.reset_index(drop=True)
        d['file'] = candidates.loc[r,'file']
        c_df = c_df.append(d,ignore_index=True)
    c_df.columns.name = ""
    return c_df

def personality_details(candidates):
    """
    Returns dataframes of raw scores for each of the Big 5 traits.
    
    candidates - dataframe of candidates
    """
    open_df = pd.DataFrame()
    cons_df = pd.DataFrame()
    extra_df = pd.DataFrame()
    agree_df = pd.DataFrame()
    emotion_df = pd.DataFrame()
    
    for r in candidates.index:
        for t in range(0,5):
            t_name = eval(candidates_df.loc[r,'personality'])[t]['name']
            d = pd.DataFrame(eval(candidates_df.loc[r,'personality'])[t]['children'])[['name','raw_score']]
            d = d.set_index('name').T.reset_index(drop=True)
            d['file'] = candidates.loc[r,'file']
            
            if t_name == 'Openness':
                open_df = open_df.append(d,ignore_index=True)
            elif t_name == 'Conscientiousness':
                cons_df = cons_df.append(d,ignore_index=True)
            elif t_name == 'Extraversion':
                extra_df = extra_df.append(d,ignore_index=True)
            elif t_name == 'Agreeableness':
                agree_df = agree_df.append(d,ignore_index=True)
            elif t_name == 'Emotional range':
                emotion_df = emotion_df.append(d,ignore_index=True)
                
    open_df.columns.name = ""
    cons_df.columns.name = ""
    extra_df.columns.name = ""
    agree_df.columns.name = ""
    emotion_df.columns.name = ""
    
    return  open_df,cons_df,extra_df,agree_df,emotion_df

In [9]:
needs_df = insights_to_df('needs',candidates_df)
values_df = insights_to_df('values',candidates_df)
consumption_df = consumption(candidates_df)
all_personality_df = insights_to_df('personality',candidates_df)
openness_df, conscientiousness_df, extraversion_df, agreeableness_df, emotional_range_df = personality_details(candidates_df)

# 5. Exploratory Data Analysis

In [10]:
needs_df

,Challenge,Closeness,Curiosity,Excitement,Harmony,Ideal,Liberty,Love,Practicality,Self-expression,Stability,Structure,file
0,0.687388,0.737678,0.827021,0.553379,0.758527,0.643312,0.694281,0.691013,0.697178,0.646145,0.707389,0.694753,instagram-kevin-systrom.txt
1,0.729555,0.705474,0.834953,0.575228,0.740460,0.643764,0.698083,0.672824,0.700580,0.613063,0.695070,0.714984,facebook-mark-zuckerberg.txt
2,0.705916,0.734802,0.833490,0.580611,0.765428,0.656928,0.703968,0.696026,0.705324,0.623352,0.701928,0.709038,spotify-daniel-ek.txt
3,0.709902,0.723402,0.843319,0.544000,0.748655,0.668819,0.695866,0.696311,0.699126,0.627310,0.690413,0.701614,twitter-ev-williams.txt


In [11]:
values_df

,Conservation,Openness to change,Hedonism,Self-enhancement,Self-transcendence,file
0,0.575899,0.766610,0.633714,0.652945,0.829168,instagram-kevin-systrom.txt
1,0.598660,0.782782,0.609476,0.656638,0.829649,facebook-mark-zuckerberg.txt
2,0.586833,0.771245,0.639145,0.661032,0.830876,spotify-daniel-ek.txt
3,0.556791,0.775472,0.623224,0.655128,0.826246,twitter-ev-williams.txt


In [12]:
consumption_df

,Health & Activity Preferences,Environmental Concern Preferences,Entrepreneurship Preferences,Movie Preferences,Reading Preferences,Volunteering Preferences,file
0,[outdoor],[concerned_environment],[start_business],"[movie_adventure, movie_musical, movie_histori...","[read_frequency, books_non_fiction, books_auto...",[volunteer],instagram-kevin-systrom.txt
1,[outdoor],[concerned_environment],[start_business],"[movie_adventure, movie_historical, movie_scie...","[read_frequency, books_non_fiction, books_fina...",[volunteer],facebook-mark-zuckerberg.txt
2,[outdoor],[concerned_environment],[],"[movie_adventure, movie_musical, movie_histori...","[read_frequency, books_non_fiction, books_auto...",[volunteer],spotify-daniel-ek.txt
3,[outdoor],[concerned_environment],[start_business],"[movie_adventure, movie_historical, movie_scie...","[read_frequency, books_non_fiction, books_fina...",[volunteer],twitter-ev-williams.txt


In [13]:
all_personality_df

,Openness,Conscientiousness,Extraversion,Agreeableness,Emotional range,file
0,0.800396,0.627016,0.523315,0.721635,0.497048,instagram-kevin-systrom.txt
1,0.795236,0.660267,0.512500,0.697280,0.497587,facebook-mark-zuckerberg.txt
2,0.791975,0.574534,0.455835,0.704284,0.473716,spotify-daniel-ek.txt
3,0.818073,0.633957,0.528237,0.721584,0.506147,twitter-ev-williams.txt


In [14]:
openness_df

,Adventurousness,Artistic interests,Emotionality,Imagination,Intellect,Authority-challenging,file
0,0.519717,0.727920,0.669370,0.709328,0.712264,0.628538,instagram-kevin-systrom.txt
1,0.541532,0.691450,0.643814,0.691068,0.722693,0.623718,facebook-mark-zuckerberg.txt
2,0.529975,0.716827,0.678005,0.730441,0.736404,0.647658,spotify-daniel-ek.txt
3,0.536035,0.733576,0.670893,0.746914,0.741595,0.667499,twitter-ev-williams.txt


In [15]:
conscientiousness_df

,Achievement striving,Cautiousness,Dutifulness,Orderliness,Self-discipline,Self-efficacy,file
0,0.675763,0.615348,0.680129,0.440515,0.530816,0.697531,instagram-kevin-systrom.txt
1,0.728193,0.613682,0.681686,0.449990,0.574116,0.761576,facebook-mark-zuckerberg.txt
2,0.686923,0.588714,0.682981,0.429911,0.521362,0.723693,spotify-daniel-ek.txt
3,0.699108,0.590725,0.683971,0.436252,0.530293,0.722200,twitter-ev-williams.txt


In [16]:
extraversion_df

,Activity level,Assertiveness,Cheerfulness,Excitement-seeking,Outgoing,Gregariousness,file
0,0.534139,0.633161,0.583899,0.517096,0.494154,0.375204,instagram-kevin-systrom.txt
1,0.616067,0.702512,0.587455,0.548125,0.557755,0.428105,facebook-mark-zuckerberg.txt
2,0.538444,0.631072,0.571256,0.544202,0.492812,0.362288,spotify-daniel-ek.txt
3,0.570366,0.646632,0.580420,0.535335,0.514934,0.370557,twitter-ev-williams.txt


In [17]:
agreeableness_df

,Altruism,Cooperation,Modesty,Uncompromising,Sympathy,Trust,file
0,0.732964,0.655640,0.485216,0.697863,0.711541,0.598488,instagram-kevin-systrom.txt
1,0.743622,0.646228,0.450045,0.674064,0.719749,0.639157,facebook-mark-zuckerberg.txt
2,0.744824,0.665624,0.482940,0.705629,0.738680,0.613193,spotify-daniel-ek.txt
3,0.752197,0.669407,0.454430,0.683956,0.753108,0.631402,twitter-ev-williams.txt


In [18]:
emotional_range_df

,Fiery,Prone to worry,Melancholy,Immoderation,Self-consciousness,Susceptible to stress,file
0,0.437068,0.532793,0.481805,0.518504,0.576679,0.476039,instagram-kevin-systrom.txt
1,0.410326,0.532186,0.441051,0.498628,0.552278,0.429566,facebook-mark-zuckerberg.txt
2,0.422803,0.555647,0.493483,0.525057,0.608136,0.469477,spotify-daniel-ek.txt
3,0.444969,0.567656,0.477213,0.497748,0.595427,0.477262,twitter-ev-williams.txt


Merge all datasets so it is easy to extract information:

# 6. IBM 1-3-9

The IBM's 139 practices are taken as a basis to determine the qualified matches:

<img src="https://upload.wikimedia.org/wikipedia/commons/1/1d/IBM_values.png" alt="IBM 139" style="width: 700px; height: 300px;"/>

- Dedication to every client's success requires skills: Curiosity, Practicality, Self-transcendence.
- Innovation that matters for our company and for the world requires skills: Self-enhancement, Imagination, Intellect.
- Trust and personal responsibility in all relationships requires skills: Trust, Dutifulness, Cooperation. 

In [19]:
def company_match(candidates):
    """ 
    Calculates the match based on company values. Returns Euclidian distance based similarity. 
    
    candidates - dateframe of candidates 
    
    """
    
    company_skills=['Curiosity', 'Practicality', 'Self-transcendence','Self-enhancement','Imagination', 'Intellect','Trust', 'Dutifulness', 'Cooperation']

    s = np.array([0.85]*len(company_skills))
    dist = np.linalg.norm(np.array(candidates[company_skills]) - s)
    similarity = 1/(1+dist)
    
    return similarity

Based on the interests of candidates managers will have an opportunity to recommend facilities and activities at IBM that candidates may enjoy:

In [20]:
def recommend_activities(activity):
    recommendation = "Candidate would like to know about {}."
    lst = []
    if set(['gym_membership']).intersection(activity['Health & Activity Preferences']):
        lst.append('on-site gym')
    if set(['eat_out']).intersection(activity['Health & Activity Preferences']):
        lst.append('cafeteria')
    if set(['read_frequency']).intersection(activity['Reading Preferences']):
        lst.append('on-site library')
    if set(['start_business']).intersection(activity['Entrepreneurship Preferences']):
        lst.append('Area 631')
    if set(['volunteer']).intersection(activity['Volunteering Preferences']):
        lst.append('ECF')
    if len(activity['Movie Preferences']) >= 1:
        lst.append('Amphitheatre movie days')
        
    return recommendation.format(", ".join(lst))

In [21]:
consumption_df['Recommend Activities'] = consumption_df.apply(recommend_activities,axis = 1)

In [22]:
consumption_df[['file','Recommend Activities']]

,file,Recommend Activities
0,instagram-kevin-systrom.txt,Candidate would like to know about on-site lib...
1,facebook-mark-zuckerberg.txt,Candidate would like to know about on-site lib...
2,spotify-daniel-ek.txt,Candidate would like to know about on-site lib...
3,twitter-ev-williams.txt,Candidate would like to know about on-site lib...


In [23]:
consumption_df.loc[0,'Recommend Activities']

'Candidate would like to know about on-site library, Area 631, ECF, Amphitheatre movie days.'

# 7. Role and Team Match

The skills indicated below were chosen based on the analysis of job descriptions.

***CHOOSE 5 ESSENTIAL SKILLS FOR YOUR TEAM:***
* Strong work values - Dependability, honesty, self-confidence and a positive attitude.
* Perseverance and motivation
* Enthusiasm
* Communication 
* Negotiation and persuasion
* Teamwork
* Leadership skills
* Confidence
* Self-management - Multitasking, adapting to changes, and flexibility.
* Detail oriented
* Ability to work under pressure
* Willingness to learn 
* Thinking skills - Problem solving, analytical skills, and decision making.
* Creativity

In [24]:
body = client_0f8d8b468baa4efa9c29025ebc262fe5.get_object(Bucket='insideout-donotdelete-pr-arlsgjghrogjm7',Key='Describe_Team.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

describe_team = pd.read_csv(body)
describe_team.head()

,Essential Skills,Extraversion,Conscientiousness,Agreeableness,Openness,Emotional range,Achievement striving,Cautiousness,Dutifulness,Orderliness,...,Modesty,Activity level,Assertiveness,Cheerfulness,Gregariousness,Susceptible to stress,Self-consciousness,Intellect,Imagination,Authority-challenging
0,Strong work values,NaN,High,High,NaN,Low,NaN,NaN,High,NaN,...,NaN,NaN,NaN,High,NaN,NaN,Low,NaN,NaN,NaN
1,Perseverance and motivation,High,High,High,NaN,Low,High,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Enthusiasm,High,High,High,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,High,NaN,NaN,NaN,NaN,NaN,NaN
3,Communication,High,NaN,High,NaN,High,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,High,NaN,NaN,NaN,NaN,NaN
4,Negotiation and persuasion,High,High,High,NaN,NaN,High,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
def team_match(skills):
    description = describe_team.copy()
    description.replace({'High':1,'Low':-1},inplace=True)
    
    dfs = [all_personality_df, openness_df, conscientiousness_df,extraversion_df,agreeableness_df,emotional_range_df]
    df = reduce(lambda left,right: pd.merge(left,right,on='file'), dfs)
    
    l = ['file']
    
    for skill in skills:
        skill_df =  description.loc[description['Essential Skills'] == skill]
        skill_df = skill_df[[col for col in skill_df.columns if skill_df[col].notnull().values]]
        lows = skill_df[skill_df == -1].count().sum()
        traits = list(skill_df.iloc[:,1:].columns)
        
        df[skill+' scores'] = ((df[traits]*skill_df[traits].values[0]).apply(np.mean,axis=1) + lows/len(traits))
        l.append(skill + ' scores')
        
    match_result = df.copy()[l]
    match_result['team_match'] = match_result[l[1:]].apply(np.mean,axis=1)
                                                                    
    return match_result

In [26]:
skill_list = ['Teamwork','Ability to work under pressure','Thinking skills','Enthusiasm','Communication']

In [27]:
def match(skills):
    c_dfs = [needs_df, values_df, openness_df, conscientiousness_df, extraversion_df, agreeableness_df, emotional_range_df]
    company_match_final = reduce(lambda left,right: pd.merge(left,right,on='file'), c_dfs)
    
    company_match_final['company_match'] = company_match_final.apply(company_match,axis=1)
    company_match_final = company_match_final[['file','company_match']]
    
    team_match_final = team_match(skills)
    
    match_final = pd.merge(company_match_final,team_match_final,on='file')
    match_final['overall_match'] = (match_final['company_match'] + match_final['team_match'])/2
    match_final.sort_values(by = 'overall_match',ascending=False,inplace=True)
    
    cols = match_final.columns.tolist()
    cols_ordered = cols[0:1] + cols[-1:] + cols[1:2] + cols[-2:-1] + cols[2:-2]
    
    return match_final[cols_ordered]

In [28]:
results = match(skills = skill_list)

In [29]:
results

,file,overall_match,company_match,team_match,Teamwork scores,Ability to work under pressure scores,Thinking skills scores,Enthusiasm scores,Communication scores
3,twitter-ev-williams.txt,0.646184,0.695495,0.596873,0.674983,0.515628,0.646075,0.616050,0.531631
1,facebook-mark-zuckerberg.txt,0.643008,0.682641,0.603374,0.667925,0.548987,0.651715,0.614375,0.533868
0,instagram-kevin-systrom.txt,0.634904,0.675013,0.594795,0.668097,0.519243,0.643367,0.613966,0.529301
2,spotify-daniel-ek.txt,0.633424,0.689685,0.577163,0.648147,0.526056,0.636103,0.576477,0.499031


# 8. Compare Candidates

In [30]:
body = client_0f8d8b468baa4efa9c29025ebc262fe5.get_object(Bucket='insideout-donotdelete-pr-arlsgjghrogjm7',Key='Describe_Needs.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

describe_needs = pd.read_csv(body)
describe_needs.head()

,Need,Description
0,Challenge,"have an urge to achieve, to succeed, and to ta..."
1,Closeness,relish being connected to family and setting u...
2,Curiosity,"have a desire to discover, find out, and grow"
3,Excitement,"want to get out there and live life, have upbe..."
4,Harmony,"appreciate other people, their viewpoints and ..."


In [31]:
body = client_0f8d8b468baa4efa9c29025ebc262fe5.get_object(Bucket='insideout-donotdelete-pr-arlsgjghrogjm7',Key='Describe_Values.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
describe_values = pd.read_csv(body)
describe_values.head()

,Value,Description
0,Conservation,"emphasize self-restriction, order, and resista..."
1,Openness to change,"emphasize independent action, thought, and fee..."
2,Hedonism,seek pleasure and sensuous gratification
3,Self-enhancement,seek personal success
4,Self-transcendence,show concern for the welfare and interests of ...


In [32]:
def compare_needs_and_values(candidate_one,candidate_two,trait):
    """
    Compares either needs or values and returns a dictionary: overall similarity score, similarities, differences, and highlights.
    
    candidate_one - name of first candidate 
    candidate_two - name of second candidate
    
    trait - 'Need' or 'Value'
    
    """
    
    if trait == 'Need':
        df = needs_df.copy()
        description = describe_needs.copy()
    elif trait == 'Value':
        df = values_df.copy()
        description = describe_values.copy()
        
    skills_one = df.loc[df['file'] == candidate_one,df.columns != 'file']
    skills_two = df.loc[df['file'] == candidate_two,df.columns != 'file']
    
    dist = np.linalg.norm(np.array(skills_one) - np.array(skills_two))
    similarity = 1/(1+dist)
    
    scores = skills_one.T.squeeze()-skills_two.T.squeeze()
    scores_text = scores.apply(lambda n: 'more' if n > 0 else 'less')
    skills_text = skills_one.T.squeeze().apply(lambda n: 'likely' if n >= 0.5 else 'less likely')
    
    similarities = list(np.abs(scores).sort_values().head(2).index)
    s_text = description.loc[description[trait].isin(similarities),'Description'].reset_index(drop=True)
    similarities_text = "Both are {} to {}. Also they are {} to {}.".format(skills_text[similarities][0], s_text[0], 
                                                                                        skills_text[similarities][1], s_text[1])
    
    differences = list(np.abs(scores).sort_values(ascending=False).head(2).index)
    d_text = description.loc[description[trait].isin(differences),'Description'].reset_index(drop=True)
    differences_text = "{} is {} likely to {}. Also he/she is {} likely than {} to {}.".format(candidate_one, 
                                                                                               scores_text[differences][0], 
                                                                                               d_text[0], 
                                                                                               scores_text[differences][1], 
                                                                                               candidate_two,
                                                                                               d_text[1])
    
    candidate_one_similarity_scores = skills_one.T.squeeze()[similarities].to_dict()
    candidate_two_similarity_scores = skills_two.T.squeeze()[similarities].to_dict()
    candidate_one_difference_scores = skills_one.T.squeeze()[differences].to_dict()
    candidate_two_difference_scores = skills_two.T.squeeze()[differences].to_dict()
    
    return {'similarity_score_{}'.format(trait): similarity, 
            'candidate_one_similarities_{}'.format(trait):candidate_one_similarity_scores,
            'candidate_two_similarities_{}'.format(trait):candidate_two_similarity_scores,
            'candidate_one_differences_{}'.format(trait):candidate_one_difference_scores,
            'candidate_two_differences_{}'.format(trait):candidate_two_difference_scores,
            'similarities_text_{}'.format(trait): similarities_text,
            'differences_text_{}'.format(trait): differences_text}

In [33]:
compare_needs_and_values('twitter-ev-williams.txt','spotify-daniel-ek.txt','Value')

{'candidate_one_differences_Value': {'Conservation': 0.5567907796003213,
  'Hedonism': 0.6232244708678959},
 'candidate_one_similarities_Value': {'Openness to change': 0.7754715206108314,
  'Self-transcendence': 0.8262458627083903},
 'candidate_two_differences_Value': {'Conservation': 0.5868332525375718,
  'Hedonism': 0.6391453328660812},
 'candidate_two_similarities_Value': {'Openness to change': 0.771244813169808,
  'Self-transcendence': 0.8308763536472082},
 'differences_text_Value': 'twitter-ev-williams.txt is less likely to emphasize self-restriction, order, and resistance to change. Also he/she is less likely than spotify-daniel-ek.txt to seek pleasure and sensuous gratification .',
 'similarities_text_Value': 'Both are likely to emphasize independent action, thought, and feeling, as well as a readiness for new experiences. Also they are likely to show concern for the welfare and interests of others.',
 'similarity_score_Value': 0.96611449041481512}

In [34]:
body = client_0f8d8b468baa4efa9c29025ebc262fe5.get_object(Bucket='insideout-donotdelete-pr-arlsgjghrogjm7',Key='Describe_Personalities_HL.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

describe_personalities_hl = pd.read_csv(body)
describe_personalities_hl.fillna(value='',inplace=True)
describe_personalities_hl.head()

,Personalities,Agreeableness High,Conscientiousness High,Extraversion High,Emotional range High,Openness High,Agreeableness Low,Conscientiousness Low,Extraversion Low,Emotional range Low,Openness Low
0,Agreeableness High,,cooperative,"communicative, enthusiastic",,"tactful, idealistic",,,,"flexible, patient",
1,Conscientiousness High,cooperative,,"purposeful, competitive",particular,"perfectionistic, analytical",,,punctual,"self-disciplined, logical, decisive, rational",
2,Extraversion High,,"purposeful, competitive",,wordy,"expressive, witty",bossy,,,confident,
3,Emotional range High,,particular,wordy,,excitable,,,,,
4,Openness High,"tactful, idealistic","perfectionistic, analytical","expressive, witty",excitable,,,,self-examining,"creative, intellectual, insightful",


In [38]:
def compare_personality(candidate_one,candidate_two):
    """
    Compares personalities and returns a dictionary: overall similarity score, similarities, differences, and highlights.
    
    candidate_one - name of first candidate 
    candidate_two - name of second candidate
    
    """
    trait='Personality'
    
    df = all_personality_df.copy()
    description_hl = describe_personalities_hl.copy()
    
    skills_one = df.loc[df['file'] == candidate_one,df.columns != 'file']
    skills_two = df.loc[df['file'] == candidate_two,df.columns != 'file']
    
    dist = np.linalg.norm(np.array(skills_one) - np.array(skills_two))
    similarity = 1/(1+dist)
    
    scores_one = skills_one.T.squeeze()
    scores_one_hl = scores_one.apply(lambda n: 'High' if n >= 0.5 else 'Low')
    cols_one = list(scores_one_hl.index + ' ' + scores_one_hl.values)
     
    scores_two = skills_two.T.squeeze()
    scores_two_hl = scores_two.apply(lambda n: 'High' if n >= 0.5 else 'Low')
    cols_two = list(scores_two_hl.index + ' ' + scores_two_hl.values)

    scores = scores_one_hl == scores_two_hl
    differences = list(scores[scores == False].index)
    
    d = {'similarity score': similarity,'differences': differences,'similarities':[]}

    if differences != []:
        differences_one = list(scores_one_hl[scores_one_hl.index.isin(differences)].index + ' ' + scores_one_hl[scores_one_hl.index.isin(differences)].values)
        differences_two = list(scores_two_hl[scores_two_hl.index.isin(differences)].index + ' ' + scores_two_hl[scores_two_hl.index.isin(differences)].values)
        candidate_one_traits = description_hl.copy().loc[description_hl['Personalities'].isin(differences_one),cols_one].values[0]
        candidate_two_traits = description_hl.copy().loc[description_hl['Personalities'].isin(differences_two),cols_two].values[0]
        candidate_text = "{} is {}."
        d['candidate_one_text'] = candidate_text.format(candidate_one,", ".join(candidate_one_traits[candidate_one_traits !=""]))
        d['candidate_two_text'] = candidate_text.format(candidate_two,", ".join(candidate_two_traits[candidate_two_traits !=""]))
        
    else:
        scores_similar = scores_one - scores_two
        scores_text = scores_similar.apply(lambda n: 'more' if n > 0 else 'less')
        similarities = list(np.abs(scores_similar).sort_values(ascending=False).head(2).index)
        similar_traits = list(scores_one_hl[similarities].index + ' ' +scores_one_hl[similarities].values)
        
        cols = list(set(cols_one).difference(set(similar_traits)))
        
        similar_text_one = description_hl.copy().loc[description_hl['Personalities'].isin([similar_traits[0]]),cols].values[0]
        similar_text_one = similar_text_one[similar_text_one != ""]
        similar_text_two = description_hl.copy().loc[description_hl['Personalities'].isin([similar_traits[1]]),cols].values[0]
        similar_text_two = similar_text_two[similar_text_two != ""]
        
        candidate_text = "Both candidates have similar personalities, however {} is {} {} than {}. Also, he/she is {} {}.".format(candidate_one,
                                                                                                                                  scores_text[similarities[0]],
                                                                                                                                  similar_text_one[0], 
                                                                                                                                  candidate_two,
                                                                                                                                  scores_text[similarities[1]],
                                                                                                                                  similar_text_two[0])
        d['similarity_text'] = candidate_text
        d['similarities'] = similarities
        
    d['candidate_one_scores'] = scores_one.to_dict()
    d['candidate_two_scores'] = scores_two.to_dict()
    
    return d

In [39]:
compare_personality('facebook-mark-zuckerberg.txt','instagram-kevin-systrom.txt')

{'candidate_one_scores': {'Agreeableness': 0.6972800956175585,
  'Conscientiousness': 0.6602665115306775,
  'Emotional range': 0.49758705115546586,
  'Extraversion': 0.512499588728126,
  'Openness': 0.7952364136410419},
 'candidate_two_scores': {'Agreeableness': 0.7216353783081719,
  'Conscientiousness': 0.6270157752629488,
  'Emotional range': 0.4970479582260236,
  'Extraversion': 0.5233151593653144,
  'Openness': 0.8003957618452753},
 'differences': [],
 'similarities': ['Conscientiousness', 'Agreeableness'],
 'similarity score': 0.95884041597472125,
 'similarity_text': 'Both candidates have similar personalities, however facebook-mark-zuckerberg.txt is more perfectionistic, analytical than instagram-kevin-systrom.txt. Also, he/she is less tactful, idealistic.'}

# END